In [2]:
import os

if os.getenv("COLAB_RELEASE_TAG"): # If running in Google Colab:
  !mkdir -p Sources
  !wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/01.02-Execution-Spaces/Sources/ach.h -nv -O Sources/ach.h

2026-01-27 09:03:07 URL:https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/01.02-Execution-Spaces/Sources/ach.h [2893/2893] -> "Sources/ach.h" [1]


In [3]:
%%writefile Sources/cpu-cooling.cpp

#include <cstdio>
#include <vector>

int main() {
    float k = 0.5;
    float ambient_temp = 20;
    std::vector<float> temp{ 42, 24, 50 };

    std::printf("step  temp[0]  temp[1]  temp[2]\n");
    for (int step = 0; step < 3; step++) {
        for (int i = 0; i < temp.size(); i++) {
            float diff = ambient_temp - temp[i];
            temp[i] = temp[i] + k * diff;
        }

        std::printf("%d     %.2f    %.2f    %.2f\n", step, temp[0], temp[1], temp[2]);
    }
}

Writing Sources/cpu-cooling.cpp


In [4]:
!g++ Sources/cpu-cooling.cpp -o /tmp/a.out # compile the code
!/tmp/a.out # run the executable

step  temp[0]  temp[1]  temp[2]
0     31.00    22.00    35.00
1     25.50    21.00    27.50
2     22.75    20.50    23.75


In [5]:
!nvcc -x cu -arch=native Sources/cpu-cooling.cpp -o /tmp/a.out # compile the code
!/tmp/a.out # run the executable

step  temp[0]  temp[1]  temp[2]
0     31.00    22.00    35.00
1     25.50    21.00    27.50
2     22.75    20.50    23.75


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [6]:
%%writefile Sources/gpu-cooling.cpp

#include <algorithm>
#include <cstdio>
#include <vector>

int main() {
    float k = 0.5;
    float ambient_temp = 20;
    std::vector<float> temp{ 42, 24, 50 };
    auto transformation = [=] (float temp) { return temp + k * (ambient_temp - temp); };

    std::printf("step  temp[0]  temp[1]  temp[2]\n");
    for (int step = 0; step < 3; step++) {
        std::transform(temp.begin(), temp.end(), temp.begin(), transformation);
        std::printf("%d     %.2f    %.2f    %.2f\n", step, temp[0], temp[1], temp[2]);
    }
}

Writing Sources/gpu-cooling.cpp


In [7]:
!nvcc Sources/gpu-cooling.cpp -x cu -arch=native -o /tmp/a.out # compile the code
!/tmp/a.out # run the executable

step  temp[0]  temp[1]  temp[2]
0     31.00    22.00    35.00
1     25.50    21.00    27.50
2     22.75    20.50    23.75


In [8]:
%%writefile Sources/thrust-cooling.cpp

#include <thrust/execution_policy.h>
#include <thrust/universal_vector.h>
#include <thrust/transform.h>
#include <cstdio>

int main() {
    float k = 0.5;
    float ambient_temp = 20;
    thrust::universal_vector<float> temp{ 42, 24, 50 };
    auto transformation = [=] __host__ __device__ (float temp) { return temp + k * (ambient_temp - temp); };

    std::printf("step  temp[0]  temp[1]  temp[2]\n");
    for (int step = 0; step < 3; step++) {
        thrust::transform(thrust::device, temp.begin(), temp.end(), temp.begin(), transformation);
        std::printf("%d     %.2f    %.2f    %.2f\n", step, temp[0], temp[1], temp[2]);
    }
}

Writing Sources/thrust-cooling.cpp


In [9]:
!nvcc --extended-lambda Sources/thrust-cooling.cpp -x cu -arch=native -o /tmp/a.out # compile the code
!/tmp/a.out # run the executable

step  temp[0]  temp[1]  temp[2]
0     31.00    22.00    35.00
1     25.50    21.00    27.50
2     22.75    20.50    23.75


In [10]:
%%writefile Sources/no-magic-execution-space-changes.cpp
#include "ach.h"

int main() {
  // TODO: Replace ??? with CPU or GPU
  ach::where_am_I("CPU");

  thrust::universal_vector<int> vec{1};
  thrust::for_each(thrust::device, vec.begin(), vec.end(),
                   [] __host__ __device__(int) { ach::where_am_I("GPU"); });

  thrust::for_each(thrust::host, vec.begin(), vec.end(),
                   [] __host__ __device__(int) { ach::where_am_I("CPU"); });

  ach::where_am_I("CPU");
}

Writing Sources/no-magic-execution-space-changes.cpp


In [11]:
!nvcc -o /tmp/a.out --extended-lambda Sources/no-magic-execution-space-changes.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

Correct! The function is invoked on CPU
Correct! The function is invoked on GPU
Correct! The function is invoked on CPU
Correct! The function is invoked on CPU
